In [1]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
import re
from itertools import product

In [2]:
def get_details(link):
    haha = req.get(link)
    html_content = haha.text
    soup = BeautifulSoup(html_content, 'html.parser')
    title_raw = soup.find('h1', id="course_preview_title").text.strip()
    
    # Modified regex pattern to capture the entire title
    match = re.match(r'(\S+\s\d+[A-Za-z]?\s*)(.+?)(?:\s*-\s*\w+)?(?:\s*\(\d+\s+credits?\))?$', title_raw)
    if match:
        course_code = match.group(1)  # e.g., "MUS 109"
        course_title = match.group(2).strip()  # Title text
    else:
        course_code = ""
        course_title = ""
    
    h1_element = soup.find('h1', id='course_preview_title')
    hours_list = []
    for sibling in h1_element.find_next_siblings():
        if sibling.name == 'em':
            hours_list.append(sibling.text.strip())
        elif sibling.name == 'br':
            break  # Stop when we hit a <br> tag
    hours = ', '.join(hours_list)
    
    #this works above do not touch
    #prereq
    semester = "Not Provided"
    em_tags = soup.find_all('em')
    prerequisites = []
    other_prereq = []
    coreq = []
    other_coreq = []
    for i, em in enumerate(em_tags):
        if "Prerequisite(s):" in em.text:
            # Start collecting <a> tags and subsequent text
            for sibling in em.find_all_next():
                if sibling.name == 'em' and sibling.text.strip() == "Taught:":
                    break
                if sibling.name == 'em' and sibling.text.strip() == "Co-requisite(s):":
                    break
                if sibling.name == 'em' and sibling.text.strip() == "Prerequisite OR Co-requisite:":
                    break
                if sibling.name == 'em' and sibling.text.strip() == "Repeatable:":
                    break
                if "Not open" in sibling.text:
                    break
                elif sibling.name == 'a':
                    prerequisites.append(sibling.text.strip())  # Add <a> text     
                elif sibling.name == 'em':
                    other_prereq.append(sibling.text.strip())  # Add <a> text 
        if "Co-requisite(s):" in em.text:
             for sibling in em.find_all_next():
                if "Not open" in sibling.text:
                    break
                if sibling.name == 'em' and sibling.text.strip() == "Taught:":
                    break
                elif sibling.name == 'a':
                    coreq.append(sibling.text.strip())  # Add <a> text     
                elif sibling.name == 'em':
                    other_coreq.append(sibling.text.strip())  # Add <a> tex
        if "Prerequisite OR Co-requisite:" in em.text:
            for sibling in em.find_next_siblings():
                if sibling.name == 'em' and sibling.text.strip() == "Taught:":
                    break
                elif sibling.name == 'a':
                    course = sibling.text.strip()
                    prerequisites.append(course)
                    coreq.append(course)
    #this works below(do not touch)
    for i, em in enumerate(em_tags):
        if "Taught:" in em.text:
            for sibling in em.find_all_next():
                text = sibling.text.strip()
                if text:  # Check if the stripped text is not empty
                    semester = text
                if sibling.name == 'br':
                    break
    prerequisites = list(dict.fromkeys(prerequisites))
    coreq = list(dict.fromkeys(coreq))
    return course_code.strip(), course_title, hours, prerequisites,coreq,semester,other_prereq,other_coreq

In [3]:
print(get_details("http://onlinecatalog.nku.edu/preview_course_nopop.php?catoid=23&coid=62016"))

('DSC 200', 'Data Wrangling', 'Hours:, 3 classroom + 0 lab/studio', ['INF 286', 'STA 205', 'STA 205R', 'STA 250', 'INF 120', 'CSC 260', 'DSC 101', 'INF 282'], ['DSC 101', 'INF 282'], 'Fall', ['INF 286\xa0, one of STA 205\xa0\xa0or\xa0STA 205R\xa0\xa0or STA 250\xa0, and an introductory programming course (e.g., INF 120\xa0, CSC 260\xa0).'], [])


In [4]:
print(get_details("http://onlinecatalog.nku.edu/preview_course_nopop.php?catoid=9&coid=17832"))

('ACC 200', 'Principles of Accounting I', 'Hours:, 3 classroom + 0 lab/studio', ['BIS 101'], ['ACC 200L', 'Back to Top', 'Print-Friendly Page (opens a new window)', 'Add to Portfolio (opens a new window)', 'catalogs', 'Modern Campus Catalog™'], 'Not Provided', ['Sophomore standing; completion of one college level mathematics course; completion of computer competence requirement as determined by student’s major, or completion of BIS 101\xa0\xa0or INF 101.'], ['ACC 200L\xa0.', '', 'Repeatable:', 'No', '', ''])


In [5]:
print(get_details("http://onlinecatalog.nku.edu/preview_course_nopop.php?catoid=11&coid=24409"))

('INF 120', 'Elementary Programming', 'Hours:, 3 classroom + 0 lab/studio', [], [], 'Fall and spring', [], [])


In [6]:
print(get_details("http://onlinecatalog.nku.edu/preview_course_nopop.php?catoid=9&coid=18072#:~:text=Co-requisite(s):%20BIO%20150L%20.%20Taught:%20Fall%20and%20spring%20The%20chemistry"))

('BIO 150', 'Introduction to Biology I', 'Hours:, 3 classroom + 0 lab/studio', ['MAHD 099', 'MAT 109'], ['BIO 150L'], 'Fall and spring', ['A minimum ACT score of 22 in math or completion of MAHD 099\xa0\xa0with a C- or higher, or placement into MAT 109\xa0\xa0or higher, AND a minimum ACT score of 18 in ENG and 20 in Reading and 19 in each remaining division (or SAT equivalent), or completion of all precollege curriculum courses.'], ['BIO 150L\xa0.'])


In [7]:
#blank

In [8]:
#running the code below will run for all the txt files.

In [9]:
course_data = []
i = 2814
with open('filtered_file.txt', 'r') as file:
    urls = file.readlines()
    for url in urls:
        url = url.strip()
        course_code, course_title, hours, prerequisites,coreq,semester,other_prereq,other_coreq  = get_details(url)
        course_data.append({
            'Course_Code': course_code,
            'Course_Title': course_title,
            'Hours': hours,
            'Prerequisites': prerequisites,  # Join prerequisites as a string
            'Coreq': coreq,
            'Semester': semester,
            'Other_Pre-req':' '.join(other_prereq),
            'Other_Co-req': ','.join(other_coreq)
        })
        print("Done for cycle:" , i)
        i -=1

Done for cycle: 2814
Done for cycle: 2813
Done for cycle: 2812
Done for cycle: 2811
Done for cycle: 2810
Done for cycle: 2809
Done for cycle: 2808
Done for cycle: 2807
Done for cycle: 2806
Done for cycle: 2805
Done for cycle: 2804
Done for cycle: 2803
Done for cycle: 2802
Done for cycle: 2801
Done for cycle: 2800
Done for cycle: 2799
Done for cycle: 2798
Done for cycle: 2797
Done for cycle: 2796
Done for cycle: 2795
Done for cycle: 2794
Done for cycle: 2793
Done for cycle: 2792
Done for cycle: 2791
Done for cycle: 2790
Done for cycle: 2789
Done for cycle: 2788
Done for cycle: 2787
Done for cycle: 2786
Done for cycle: 2785
Done for cycle: 2784
Done for cycle: 2783
Done for cycle: 2782
Done for cycle: 2781
Done for cycle: 2780
Done for cycle: 2779
Done for cycle: 2778
Done for cycle: 2777
Done for cycle: 2776
Done for cycle: 2775
Done for cycle: 2774
Done for cycle: 2773
Done for cycle: 2772
Done for cycle: 2771
Done for cycle: 2770
Done for cycle: 2769
Done for cycle: 2768
Done for cycl

Done for cycle: 2423
Done for cycle: 2422
Done for cycle: 2421
Done for cycle: 2420
Done for cycle: 2419
Done for cycle: 2418
Done for cycle: 2417
Done for cycle: 2416
Done for cycle: 2415
Done for cycle: 2414
Done for cycle: 2413
Done for cycle: 2412
Done for cycle: 2411
Done for cycle: 2410
Done for cycle: 2409
Done for cycle: 2408
Done for cycle: 2407
Done for cycle: 2406
Done for cycle: 2405
Done for cycle: 2404
Done for cycle: 2403
Done for cycle: 2402
Done for cycle: 2401
Done for cycle: 2400
Done for cycle: 2399
Done for cycle: 2398
Done for cycle: 2397
Done for cycle: 2396
Done for cycle: 2395
Done for cycle: 2394
Done for cycle: 2393
Done for cycle: 2392
Done for cycle: 2391
Done for cycle: 2390
Done for cycle: 2389
Done for cycle: 2388
Done for cycle: 2387
Done for cycle: 2386
Done for cycle: 2385
Done for cycle: 2384
Done for cycle: 2383
Done for cycle: 2382
Done for cycle: 2381
Done for cycle: 2380
Done for cycle: 2379
Done for cycle: 2378
Done for cycle: 2377
Done for cycl

Done for cycle: 2032
Done for cycle: 2031
Done for cycle: 2030
Done for cycle: 2029
Done for cycle: 2028
Done for cycle: 2027
Done for cycle: 2026
Done for cycle: 2025
Done for cycle: 2024
Done for cycle: 2023
Done for cycle: 2022
Done for cycle: 2021
Done for cycle: 2020
Done for cycle: 2019
Done for cycle: 2018
Done for cycle: 2017
Done for cycle: 2016
Done for cycle: 2015
Done for cycle: 2014
Done for cycle: 2013
Done for cycle: 2012
Done for cycle: 2011
Done for cycle: 2010
Done for cycle: 2009
Done for cycle: 2008
Done for cycle: 2007
Done for cycle: 2006
Done for cycle: 2005
Done for cycle: 2004
Done for cycle: 2003
Done for cycle: 2002
Done for cycle: 2001
Done for cycle: 2000
Done for cycle: 1999
Done for cycle: 1998
Done for cycle: 1997
Done for cycle: 1996
Done for cycle: 1995
Done for cycle: 1994
Done for cycle: 1993
Done for cycle: 1992
Done for cycle: 1991
Done for cycle: 1990
Done for cycle: 1989
Done for cycle: 1988
Done for cycle: 1987
Done for cycle: 1986
Done for cycl

Done for cycle: 1641
Done for cycle: 1640
Done for cycle: 1639
Done for cycle: 1638
Done for cycle: 1637
Done for cycle: 1636
Done for cycle: 1635
Done for cycle: 1634
Done for cycle: 1633
Done for cycle: 1632
Done for cycle: 1631
Done for cycle: 1630
Done for cycle: 1629
Done for cycle: 1628
Done for cycle: 1627
Done for cycle: 1626
Done for cycle: 1625
Done for cycle: 1624
Done for cycle: 1623
Done for cycle: 1622
Done for cycle: 1621
Done for cycle: 1620
Done for cycle: 1619
Done for cycle: 1618
Done for cycle: 1617
Done for cycle: 1616
Done for cycle: 1615
Done for cycle: 1614
Done for cycle: 1613
Done for cycle: 1612
Done for cycle: 1611
Done for cycle: 1610
Done for cycle: 1609
Done for cycle: 1608
Done for cycle: 1607
Done for cycle: 1606
Done for cycle: 1605
Done for cycle: 1604
Done for cycle: 1603
Done for cycle: 1602
Done for cycle: 1601
Done for cycle: 1600
Done for cycle: 1599
Done for cycle: 1598
Done for cycle: 1597
Done for cycle: 1596
Done for cycle: 1595
Done for cycl

Done for cycle: 1250
Done for cycle: 1249
Done for cycle: 1248
Done for cycle: 1247
Done for cycle: 1246
Done for cycle: 1245
Done for cycle: 1244
Done for cycle: 1243
Done for cycle: 1242
Done for cycle: 1241
Done for cycle: 1240
Done for cycle: 1239
Done for cycle: 1238
Done for cycle: 1237
Done for cycle: 1236
Done for cycle: 1235
Done for cycle: 1234
Done for cycle: 1233
Done for cycle: 1232
Done for cycle: 1231
Done for cycle: 1230
Done for cycle: 1229
Done for cycle: 1228
Done for cycle: 1227
Done for cycle: 1226
Done for cycle: 1225
Done for cycle: 1224
Done for cycle: 1223
Done for cycle: 1222
Done for cycle: 1221
Done for cycle: 1220
Done for cycle: 1219
Done for cycle: 1218
Done for cycle: 1217
Done for cycle: 1216
Done for cycle: 1215
Done for cycle: 1214
Done for cycle: 1213
Done for cycle: 1212
Done for cycle: 1211
Done for cycle: 1210
Done for cycle: 1209
Done for cycle: 1208
Done for cycle: 1207
Done for cycle: 1206
Done for cycle: 1205
Done for cycle: 1204
Done for cycl

Done for cycle: 852
Done for cycle: 851
Done for cycle: 850
Done for cycle: 849
Done for cycle: 848
Done for cycle: 847
Done for cycle: 846
Done for cycle: 845
Done for cycle: 844
Done for cycle: 843
Done for cycle: 842
Done for cycle: 841
Done for cycle: 840
Done for cycle: 839
Done for cycle: 838
Done for cycle: 837
Done for cycle: 836
Done for cycle: 835
Done for cycle: 834
Done for cycle: 833
Done for cycle: 832
Done for cycle: 831
Done for cycle: 830
Done for cycle: 829
Done for cycle: 828
Done for cycle: 827
Done for cycle: 826
Done for cycle: 825
Done for cycle: 824
Done for cycle: 823
Done for cycle: 822
Done for cycle: 821
Done for cycle: 820
Done for cycle: 819
Done for cycle: 818
Done for cycle: 817
Done for cycle: 816
Done for cycle: 815
Done for cycle: 814
Done for cycle: 813
Done for cycle: 812
Done for cycle: 811
Done for cycle: 810
Done for cycle: 809
Done for cycle: 808
Done for cycle: 807
Done for cycle: 806
Done for cycle: 805
Done for cycle: 804
Done for cycle: 803


Done for cycle: 442
Done for cycle: 441
Done for cycle: 440
Done for cycle: 439
Done for cycle: 438
Done for cycle: 437
Done for cycle: 436
Done for cycle: 435
Done for cycle: 434
Done for cycle: 433
Done for cycle: 432
Done for cycle: 431
Done for cycle: 430
Done for cycle: 429
Done for cycle: 428
Done for cycle: 427
Done for cycle: 426
Done for cycle: 425
Done for cycle: 424
Done for cycle: 423
Done for cycle: 422
Done for cycle: 421
Done for cycle: 420
Done for cycle: 419
Done for cycle: 418
Done for cycle: 417
Done for cycle: 416
Done for cycle: 415
Done for cycle: 414
Done for cycle: 413
Done for cycle: 412
Done for cycle: 411
Done for cycle: 410
Done for cycle: 409
Done for cycle: 408
Done for cycle: 407
Done for cycle: 406
Done for cycle: 405
Done for cycle: 404
Done for cycle: 403
Done for cycle: 402
Done for cycle: 401
Done for cycle: 400
Done for cycle: 399
Done for cycle: 398
Done for cycle: 397
Done for cycle: 396
Done for cycle: 395
Done for cycle: 394
Done for cycle: 393


In [10]:
course_df = pd.DataFrame(course_data)

In [ ]:
course_data = {
        'Course_Code': "MAHD 092",
        'Course_Title': "Basic Math Skills and Essential Algebra",
        'Hours': "4 classroom + 2 lab/studio",
        'Prerequisites': [],  # If prerequisites are a string already
        'Coreq': [],
        'Semester': "Fall, Spring, Summer",
        'Other_Pre-req': ' ',  # Join the list into a string
        'Other_Co-req': '' # Join the list into a string
    }

In [ ]:
course_df

In [11]:
course_df.to_csv('course_details.csv', index=False)